# Question Answering Model 

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [1]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DefaultDataCollator,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer
)
import torch

import evaluate
import collections
from tqdm.auto import tqdm
import numpy as np

import os
import re
import datetime

/mount/arbeitsdaten31/studenten1/linku/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set cache directory.

In [2]:
model_dir = '/mount/arbeitsdaten31/studenten1/linku/models'
CACHE_DIR='/mount/arbeitsdaten31/studenten1/linku/cache'
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache
env: HF_MODULES_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache
env: HF_DATASETS_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache


### arguments.py

args_input.

In [3]:
args_input_ALstrategy = 'MarginSampling'
args_input_initseed = 100 # 1000
args_input_quota = 100 # 1000
args_input_batch = 35 # 128
args_input_dataset_name = 'SQuAD'
args_input_iteration = 1
args_input_model_batch = 8 # already add in arguments.py

### load dataset

In [4]:
squad = load_dataset(args_input_dataset_name.lower())
# squad["train"] = squad["train"].shuffle(42).select(range(2000))
squad["train"] = squad["train"].select(range(3000))
squad["validation"] = squad["validation"].select(range(1000))

Found cached dataset squad (/home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 102.68it/s]


Next we will preprocess the dataset (training and evaluation data).

In [5]:
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [7]:
max_length = 384
stride = 128

# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# preprocess data
train_dataset = squad["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad["train"].column_names,
)
val_dataset = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)

Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-2a9d31b0890526df.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0f087a92e258c27b.arrow


In [8]:
len(train_dataset['example_id']) # 10138 more than before tokenizing

3074

In [9]:
len(set(train_dataset['example_id'])) # 10000 same as before tokenizing

3000

## Evaluation

In [10]:
metric = evaluate.load("squad")

In [11]:
def compute_metrics(start_logits, end_logits, val_dataset, examples):
    
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(val_dataset):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = val_dataset[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [75]:
def get_prob(trainer_qs, features, examples):
    # thinking the posibility of get rid of 'examples'
    preds, _, _ = trainer_qs.predict(features)
    start_logits, end_logits = preds

    prob_list_dict = []
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20 # TODO: if set n_best as 5, will it effect the time??
    
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    for example in tqdm(examples):
        example_id = example["id"]
        # context = example["context"]
        answers = []
        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])
        
        if len(answers) > 1:
            answers.sort(reverse=True)
            # n_best_sorted_answers = answers[:n_best]
            n_best_sorted_prob = softmax(answers)
            prob_list_dict.append(
                {'idx': example_to_features[example_id], 
                 'probs': n_best_sorted_prob}
            )

        elif example_to_features[example_id]:
            prob_list_dict.append(
                {'idx': example_to_features[example_id], 
                 'probs': np.array([0])}
            )
    
    return prob_list_dict
# move to evaluation.py

In [105]:
def get_prob_dropout(trainer_qs, features, examples, n_drop=10):
    prob_dict = {}
    for i in range(n_drop):
        preds, _, _ = trainer_qs.predict(features)
        start_logits, end_logits = preds

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        for example in tqdm(examples):
            example_id = example["id"]
            # context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])
            
            if len(answers) > 1:
                if example_to_features[example_id] not in prob_dict:
                    prob_dict[example_to_features[example_id]] = softmax(answers)
                else:
                    prob_dict[example_to_features[example_id]] += softmax(answers)
            elif example_to_features[example_id] and example_to_features[example_id] not in prob_dict:
                prob_dict[example_to_features[example_id]] = np.array([0])

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop
    return prob_dict
# move to evaluation.py

In [87]:
ans = np.array([0])
np.append(ans, 0.)

In [91]:
d = {}
d[0] = np.array([0.1, 0.02, 0.03])
d[1] = np.array([0.8, 0.7, 0.6])

In [92]:
d[0] += np.array([0.2, 0.13, 0.18])
d

{0: array([0.3 , 0.15, 0.21]), 1: array([0.8, 0.7, 0.6])}

In [96]:
for k in d.keys():
    d[k] /= 2
d

{0: array([0.15 , 0.075, 0.105]), 1: array([0.4 , 0.35, 0.3 ])}

## utils.py

In [13]:
def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data

# move to utils.py

In [14]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
# move to utils.py

## Query

In [15]:
def random_sampling_query(labeled_idxs, n):
    return np.random.choice(np.where(labeled_idxs==0)[0], n, replace=False)

In [16]:
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: probs_sorted, _ = probs.sort(descending=True)
    # deepAL+: uncertainties = probs_sorted[:, 0] - probs_sorted[:,1]
    uncertainties_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            uncertainties = d['probs'][0] - d['probs'][1]
            uncertainties_list_dict.append(
                {'idx': d['idx'], 
                 'uncertainties': uncertainties}
                 )
        elif d['idx']:
            uncertainties_list_dict.append(
                {'idx': d['idx'], 
                 'uncertainties': np.array([0])}
                 )
    
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]   
    sorted_uncertainties_dict = sorted(uncertainties_list_dict, key=lambda d: d['uncertainties']) 
    
    return unlabeled_idxs[[uncertainties_dict['idx'][0] for uncertainties_dict in sorted_uncertainties_dict[:n]]]

In [17]:
def least_confidence_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: uncertainties = probs.max(1)[0]
    confidence_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])   
    return unlabeled_idxs[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:n]]]

In [18]:
def var_ratio_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: preds = torch.max(probs, 1)[0]
    # deepAL+: uncertainties = 1.0 - preds
    confidence_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': 1 - confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )

    # deepAL+: return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]
    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'], reverse=True)
    return unlabeled_idxs[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:n]]]
# comment for the same query as LC

In [19]:
def entropy_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: log_probs = torch.log(probs)
    # deepAL+: uncertainties = (probs*log_probs).sum(1)
    entropy_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            log_probs = np.log(d['probs'])
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': (d['probs']*log_probs).sum()}
                )
        elif d['idx']:
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': np.array([0])}
                )
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] # use smallest Entropy
    sorted_entropy_dict = sorted(entropy_list_dict, key=lambda d: d['entropy'], reverse=True) # use largest Entropy, different from deepAL+
    return unlabeled_idxs[[entropy_dict['idx'][0] for entropy_dict in sorted_entropy_dict[:n]]]

# main.py

### parameters

In [24]:
NUM_QUERY = args_input_batch
NUM_INIT_LB = args_input_initseed
NUM_ROUND = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

### seed and device

In [25]:
SEED = 4666
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(3)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### start experiment

In [26]:
iteration = args_input_iteration
model_batch = args_input_model_batch

all_acc = []
acq_time = []

In [27]:
# repeate # iteration trials
while (iteration > 0): 
	iteration = iteration - 1

	## data, network, strategy
	net = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)

	## set up training
	training_args = TrainingArguments(
		output_dir=model_dir,
		evaluation_strategy="no",
		eval_steps=100,
		logging_steps=100,
		learning_rate=1e-4,
		per_device_train_batch_size=model_batch,
		per_device_eval_batch_size=model_batch, 
		gradient_accumulation_steps=1,
		num_train_epochs=3,  # max_steps will override this value
		# max_steps=1000,  # comment out if this is not wanted
		weight_decay=0.01,
		report_to="none",
	)

	## data collator for batching
	data_collator = DefaultDataCollator()

	start = datetime.datetime.now()

	## generate initial labeled pool
	n_pool = len(train_dataset)
	labeled_idxs = np.zeros(n_pool, dtype=bool)

	tmp_idxs = np.arange(n_pool)
	np.random.shuffle(tmp_idxs)
	labeled_idxs[tmp_idxs[:args_input_initseed]] = True

	run_0_labeled_idxs = np.arange(n_pool)[labeled_idxs] 
	
	## record acc performance 
	acc = np.zeros(NUM_ROUND + 1) # build 3 runs + run_0 # origin 10 runs + run_0

	## the actual trainer which performs training and evaluation
	trainer_0 = Trainer(
						model=net,
						args=training_args,
						train_dataset=train_dataset.select(indices=run_0_labeled_idxs),
						eval_dataset=val_dataset,
						tokenizer=tokenizer,
						data_collator=data_collator
						)	
		
	## print info
	print(DATA_NAME)
	# print('RANDOM SEED {}'.format(SEED))
	print(STRATEGY_NAME) # print(type(strategy).__name__)
	
	## round 0 accuracy 
	trainer_0.train() # already update to main.py
	model_saved_dir = model_dir + '/train_bert_squad_0'
	trainer_0.save_model(model_saved_dir)

	preds, _, _ = trainer_0.predict(val_dataset)
	start_logits, end_logits = preds
	acc[0] = compute_metrics(start_logits, end_logits, val_dataset, squad["validation"])['exact_match']

	trainer_qs = trainer_0

	# print('Round 0\ntesting accuracy {}'.format(acc[0]))
	# print('\n')
	
	# ## round 1 to rd
	# for rd in range(1, NUM_ROUND+1):
	# 	print('Round {}'.format(rd))
		
	# 	## query
	# 	if STRATEGY_NAME == 'RandomSampling':
	# 		q_idxs = random_sampling_query(labeled_idxs)
	# 	elif STRATEGY_NAME == 'MarginSampling':
	# 		q_idxs = margin_sampling_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'])
	# 	elif STRATEGY_NAME == 'LeastConfidence':
	# 		q_idxs = least_confidence_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'])
	# 	elif STRATEGY_NAME == 'EntropySampling':
	# 		q_idxs = entropy_query()
	# 	elif STRATEGY_NAME == 'MarginSamplingDropout':
	# 		q_idxs = margin_sampling_dropout_query()
	# 	elif STRATEGY_NAME == 'LeastConfidenceDropout':
	# 		q_idxs = least_confidence_dropout_query()
	# 	elif STRATEGY_NAME == 'EntropySamplingDropout':
	# 		q_idxs = entropy_dropout_query()
	# 	elif STRATEGY_NAME == 'VarRatio':
	# 		q_idxs = var_ratio_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'])
	# 	elif STRATEGY_NAME == 'KMeansSampling':
	# 		q_idxs = kmeans_query()
	# 	elif STRATEGY_NAME == 'KCenterGreedy':
	# 		q_idxs = kcenter_query()
	# 	# elif STRATEGY_NAME == 'KCenterGreedyPCA': # not sure
	# 	# 	q_idxs = 
	# 	elif STRATEGY_NAME == 'BALDDropout':
	# 		q_idxs = bayesian_query()
	# 	elif STRATEGY_NAME == 'MeanSTD':
	# 		q_idxs = mean_std_query()
	# 	elif STRATEGY_NAME == 'BadgeSampling':
	# 		q_idxs = badge_query()
	# 	elif STRATEGY_NAME == 'LossPredictionLoss':
	# 		# different net!
	# 		q_idxs = loss_prediction_query()
	# 	elif STRATEGY_NAME == 'CEALSampling':
	# 		# why use 'CEALSampling' in STRATEGY_NAME
	# 		q_idxs = ceal_query()
	# 	else:
	# 		raise NotImplementedError
	
	# 	## update
	# 	labeled_idxs[q_idxs] = True
	# 	run_rd_labeled_idxs = np.arange(n_pool)[labeled_idxs]
	# 	print('run_rd_labeled_idxs', len(run_rd_labeled_idxs))

	# 	trainer_rd = Trainer(
	# 					model=AutoModelForQuestionAnswering.from_pretrained(model_saved_dir).to(device),
	# 					args=training_args,
	# 					train_dataset=train_dataset.select(indices=run_rd_labeled_idxs),
	# 					eval_dataset=val_dataset,
	# 					tokenizer=tokenizer,
	# 					data_collator=data_collator
	# 					)

	# 	## train # already update to main.py
	# 	trainer_rd.train()
	# 	model_saved_dir = 'train_bert_squad_' + rd
	# 	trainer_rd.save_model(model_saved_dir)

	# 	trainer_qs = trainer_rd
	
	# 	## round rd accuracy
	# 	preds, _, _ = trainer_rd.predict(val_dataset)
	# 	start_logits, end_logits = preds
	# 	acc[rd] = compute_metrics(start_logits, end_logits, val_dataset, squad["validation"])['exact_match']
	# 	print('testing accuracy {}'.format(acc[rd]))
	# 	print('\n')

	# 	torch.cuda.empty_cache()
	
	# ## print results
	# print('SEED {}'.format(SEED))
	# print(STRATEGY_NAME)
	# print(acc)
	# all_acc.append(acc)
	
	# ## save model
	# timestamp = re.sub('\.[0-9]*','_',str(datetime.datetime.now())).replace(" ", "_").replace("-", "").replace(":","")
	# model_path = './modelpara/' + timestamp + DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input_quota)  +'.params'
	# end = datetime.datetime.now()
	# acq_time.append(round(float((end - start).seconds), 3))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_out

SQuAD
MarginSampling


/mount/arbeitsdaten31/studenten1/linku/.venv/lib64/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


100%|██████████| 1000/1000 [00:03<00:00, 251.38it/s]


## query workspace

In [28]:
trainer_qs = trainer_0

In [29]:
# get unlable data
unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
len(unlabeled_idxs)

2974

### test: query 5 data from 20 unlabeled_data

In [99]:
# smaller data
unlabeled_idxs_20 = unlabeled_idxs[20:40]
unlabeled_data_20 = train_dataset.select(unlabeled_idxs_20)
len(unlabeled_data_20)

20

In [100]:
unlabeled_idxs_20

array([20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40])

In [62]:
type(unlabeled_idxs_20)

numpy.ndarray

In [101]:
probs_list_dict_20 = get_prob(trainer_qs, unlabeled_data_20, squad['train'])
# len(probs_list_dict_20)

100%|██████████| 3000/3000 [00:00<00:00, 5851.62it/s]


In [142]:
def get_prob_dropout(trainer_qs, features, examples, n_drop=5):
    prob_dict = {}
    for_check = []
    
    for i in range(n_drop):
        preds, _, _ = trainer_qs.predict(features)
        start_logits, end_logits = preds

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            example_id = example["id"]
            # context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            if len(answers) > 1:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = softmax(answers)

                else:
                    prob_dict[example_to_features[example_id][0]] += softmax(answers)
            elif example_to_features[example_id] != []:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = np.array([0])
            if n == 0 and len(softmax(answers)) > 1:
                for_check.append(answers[:5])
                n += 1      

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop
    return prob_dict, for_check
# move to evaluation.py

In [143]:
probs_list_dict_20_dropout, for_check = get_prob_dropout(trainer_qs, unlabeled_data_20, squad['train'])

100%|██████████| 3000/3000 [00:00<00:00, 5734.84it/s]


100%|██████████| 3000/3000 [00:00<00:00, 5820.01it/s]


100%|██████████| 3000/3000 [00:00<00:00, 5825.58it/s]


100%|██████████| 3000/3000 [00:00<00:00, 6011.69it/s]


100%|██████████| 3000/3000 [00:00<00:00, 5953.54it/s]


In [144]:
for_check
# the prediction are the same

[[5.8517385, 4.0963006, 4.0622125, 5.072318, 4.9641094],
 [5.8517385, 4.0963006, 4.0622125, 5.072318, 4.9641094],
 [5.8517385, 4.0963006, 4.0622125, 5.072318, 4.9641094],
 [5.8517385, 4.0963006, 4.0622125, 5.072318, 4.9641094],
 [5.8517385, 4.0963006, 4.0622125, 5.072318, 4.9641094]]

In [120]:
probs_list_dict_20_dropout

{0: array([0.09048778, 0.01563914, 0.01511502, 0.04150419, 0.03724753,
        0.02748025, 0.02079083, 0.01467046, 0.01446176, 0.0141665 ,
        0.01355814, 0.01209789, 0.01193168, 0.01174672, 0.01135304,
        0.05076183, 0.03866458, 0.0244996 , 0.01414402, 0.01388623,
        0.0072037 , 0.00657532, 0.02396834, 0.01699181, 0.01383731,
        0.01358511, 0.01215176, 0.02045504, 0.0150912 , 0.00805651,
        0.00794189, 0.00777975, 0.01967757, 0.00775029, 0.00764003,
        0.02599083, 0.01646894, 0.00950779, 0.0093345 , 0.00834963,
        0.00442002, 0.02166077, 0.01649871, 0.00307392, 0.00280578,
        0.01611677, 0.01227592, 0.00228716, 0.00208765, 0.01525799,
        0.0116218 , 0.00736409, 0.00216529, 0.00197641, 0.00402448,
        0.00262445, 0.00258839, 0.00254827, 0.00246287, 0.00273657,
        0.0060964 , 0.00547115, 0.00403647, 0.00305389, 0.00215489,
        0.00212424, 0.00208086, 0.0110093 , 0.00838564, 0.00531351,
        0.00301167, 0.00156235, 0.00142607, 0

In [119]:
probs_list_dict_20

[{'idx': [0],
  'probs': array([0.09048778, 0.05076183, 0.04150419, 0.03866458, 0.03724753,
         0.02748025, 0.02599083, 0.0244996 , 0.02396833, 0.02166077,
         0.02079083, 0.02045504, 0.01967757, 0.01699181, 0.01649871,
         0.01646895, 0.01611677, 0.01563914, 0.01525799, 0.01511502,
         0.0150912 , 0.01467046, 0.01446176, 0.0141665 , 0.01414402,
         0.01388623, 0.01383731, 0.01358511, 0.01355814, 0.01227592,
         0.01215177, 0.01209789, 0.01193168, 0.01174672, 0.0116218 ,
         0.01135304, 0.0110093 , 0.00950779, 0.0093345 , 0.00838564,
         0.00834963, 0.00811123, 0.0080565 , 0.00794189, 0.00777975,
         0.00775029, 0.00764003, 0.00736409, 0.0072037 , 0.00657532,
         0.00617821, 0.0060964 , 0.00547115, 0.00531351, 0.00460301,
         0.00442002, 0.00403647, 0.00402448, 0.00395668, 0.0032632 ,
         0.00307392, 0.00305389, 0.00302047, 0.00301167, 0.00291913,
         0.00280578, 0.00273657, 0.00265739, 0.00262445, 0.00260896,
         0.

In [78]:
confidence_list_dict = []
for d in probs_list_dict_20:
    if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
        confidence = max(d['probs'])
        confidence_list_dict.append(
            {'idx': d['idx'], 
                'confidence': confidence}
                )
    elif d['idx']:
        confidence_list_dict.append(
            {'idx': d['idx'], 
                'confidence': np.array([0])}
                )
# deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])   
unlabeled_idxs_20[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:5]]]

array([34, 35, 39, 40, 33])

In [81]:
def get_entropy(prob_list_dict, NUM_QUERY):
    entropy_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            log_probs = np.log(d['probs'])
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': (d['probs']*log_probs).sum()}
                )
        elif d['idx']:
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': np.array([0])}
                )
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] # use smallest Entropy
    sorted_entropy_dict = sorted(entropy_list_dict, key=lambda d: d['entropy'], reverse=True) # use largest Entropy, different from deepAL+
    return unlabeled_idxs[[entropy_dict['idx'][0] for entropy_dict in sorted_entropy_dict[:NUM_QUERY]]]

In [83]:
q_idxs_20 = get_entropy(probs_list_dict_20, 5)
q_idxs_20

array([17, 19, 15, 16, 18])

In [147]:
preds, _, _ = trainer_qs.predict(unlabeled_data_20)

In [148]:
start_logits, end_logits = preds
start_logits

array([[ 0.11129025, -2.3665497 , -0.79520124, ..., -4.5936365 ,
        -4.586774  , -4.5850654 ],
       [ 0.15815008, -2.4326143 , -0.5714892 , ..., -4.6285734 ,
        -4.6285872 , -4.6248646 ],
       [ 0.09457874, -2.5953221 , -1.8137492 , ..., -4.5691795 ,
        -4.5709004 , -4.5793557 ],
       ...,
       [ 0.20229188, -2.5083427 , -2.0060809 , ..., -4.695624  ,
        -4.69039   , -4.7005043 ],
       [ 0.24428111, -2.3364782 , -0.4007522 , ..., -4.6479974 ,
        -4.64676   , -4.645563  ],
       [ 0.22122052, -2.4042737 , -1.0067688 , ..., -4.672873  ,
        -4.674313  , -4.6712456 ]], dtype=float32)

In [145]:
# predict with unlable data
preds, _, _ = trainer_qs.predict(unlabeled_data_20)

In [146]:
start_logits, end_logits = preds
start_logits

array([[ 0.11129025, -2.3665497 , -0.79520124, ..., -4.5936365 ,
        -4.586774  , -4.5850654 ],
       [ 0.15815008, -2.4326143 , -0.5714892 , ..., -4.6285734 ,
        -4.6285872 , -4.6248646 ],
       [ 0.09457874, -2.5953221 , -1.8137492 , ..., -4.5691795 ,
        -4.5709004 , -4.5793557 ],
       ...,
       [ 0.20229188, -2.5083427 , -2.0060809 , ..., -4.695624  ,
        -4.69039   , -4.7005043 ],
       [ 0.24428111, -2.3364782 , -0.4007522 , ..., -4.6479974 ,
        -4.64676   , -4.645563  ],
       [ 0.22122052, -2.4042737 , -1.0067688 , ..., -4.672873  ,
        -4.674313  , -4.6712456 ]], dtype=float32)

In [85]:
q_idxs = get_entropy(probs_list_dict, NUM_QUERY)
q_idxs

array([ 624,  621,  627,  618,  628,  626,  617,  619,  622,  623,  620,
        625, 1236, 1235,  446, 1224, 2579,  442,  445, 1237, 2577,  448,
        439,  444, 1227,  450,  447,  451, 2581,  440, 2576, 1985,  443,
       1983, 2981])